<a href="https://colab.research.google.com/github/EdWangLoDaSc/Dropout-as-a-Grid-Search_Representing-Model-Uncertainty-in-Deep-Learning/blob/main/Bootstrap%20MAP%20Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip3 install http://download.pytorch.org/whl/cu92/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
!pip install GPy
import pandas as pd
import zipfile
import urllib.request
import os
import GPy
import time
import copy
import math
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold
from torch.autograd import Variable
from torch.optim import Optimizer
from torch.optim.sgd import SGD

from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange
from google.colab import files
%config InlineBackend.figure_format = 'svg'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: torch-0.4.1-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
torch.cuda.device(0)
torch.cuda.get_device_name(torch.cuda.current_device())

'Tesla T4'

In [13]:
def to_variable(var=(), cuda=False, volatile=False):
    out = []
    for v in var:
        
        if isinstance(v, np.ndarray):
            v = torch.from_numpy(v).type(torch.FloatTensor)

        if not v.is_cuda and cuda:
            v = v.cuda()

        if not isinstance(v, Variable):
            v = Variable(v, volatile=volatile)

        out.append(v)
    return out

In [14]:
def log_gaussian_loss(output, target, sigma, no_dim):
    exponent = -0.5*(target - output)**2/sigma**2
    log_coeff = -no_dim*torch.log(sigma)
    
    return - (log_coeff + exponent).sum()


def get_kl_divergence(weights, prior, varpost):
    prior_loglik = prior.loglik(weights)
    
    varpost_loglik = varpost.loglik(weights)
    varpost_lik = varpost_loglik.exp()
    
    return (varpost_lik*(varpost_loglik - prior_loglik)).sum()


class gaussian:
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
        
    def loglik(self, weights):
        exponent = -0.5*(weights - self.mu)**2/self.sigma**2
        log_coeff = -0.5*(np.log(2*np.pi) + 2*np.log(self.sigma))
        
        return (exponent + log_coeff).sum()

In [15]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output, learn_rate, weight_decay):
        super(Net, self).__init__()
        self.layer1 = torch.nn.Linear(n_feature, n_hidden)
        self.layer2 = torch.nn.Linear(n_hidden, 2*n_output)
        
        self.loss_func = log_gaussian_loss
        self.optimizer = torch.optim.SGD(self.parameters(), lr=learn_rate, weight_decay=weight_decay)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.layer2(x)
        return x
    
    def fit(self, x, y):
        x, y = to_variable(var=(x, y), cuda=False)
        
        # reset gradient and total loss
        self.optimizer.zero_grad()
        
        output = self.forward(x)
        loss = self.loss_func(output[:, :1], y, output[:, 1:].exp(), 1)/x.shape[0]
        
        loss.backward()
        self.optimizer.step()

        return loss

In [16]:
def eval_ensemble(x, y, ensemble):
    
    x, y = to_variable(var=(x, y), cuda=False)
        
    means, stds = [], []
    for net in ensemble:
        output = net(x)
        means.append(output[:, :1, None])
        stds.append(output[:, 1:, None].exp())
            
    means, stds = torch.cat(means, 2), torch.cat(stds, 2)
    mean = means.mean(dim=2)
    std = (means.var(dim=2) + stds.mean(dim=2)**2)**0.5
    loss = log_gaussian_loss(mean, y, std, 1)/len(x)
    
    rmse = ((mean - y)**2).mean()**0.5

    return loss, rmse

In [17]:
class Net_UCI(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output, learn_rate, weight_decay):
        super(Net_UCI, self).__init__()
        self.hidden1 = torch.nn.Linear(n_feature, n_hidden)
        self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)
        self.predict = torch.nn.Linear(n_hidden, 2*n_output)
        
        self.loss_func = log_gaussian_loss
        self.optimizer = torch.optim.SGD(self.parameters(), lr=learn_rate, weight_decay=weight_decay)
        
    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = self.predict(x)
        return x
    
    def fit(self, x, y):
        x, y = to_variable(var=(x, y), cuda=False)
        
        # reset gradient and total loss
        self.optimizer.zero_grad()
        
        output = self.forward(x)
        loss = self.loss_func(output[:, :1], y, output[:, 1:].exp(), 1)/x.shape[0]
        
        loss.backward()
        self.optimizer.step()

        return loss

In [28]:
def train_ensemble(data, n_splits, num_epochs, num_nets, num_hidden, learn_rate, weight_decay, data_fraction, log_every):
    
    kf = KFold(n_splits=n_splits)
    in_dim = data.shape[1] - 1
    train_logliks, test_logliks = [], []
    train_rmses, test_rmses = [], []

    for j, idx in enumerate(kf.split(data)):
        train_index, test_index = idx

        x_train, y_train = data[train_index, :in_dim], data[train_index, in_dim:]
        x_test, y_test = data[test_index, :in_dim], data[test_index, in_dim:]

        #x_means, x_stds = x_train.mean(axis = 0), x_train.var(axis = 0)**0.5
        y_means, y_stds = y_train.mean(axis = 0), y_train.var(axis = 0)**0.5

        #x_train = (x_train - x_means)/x_stds
        #y_train = (y_train - y_means)/y_stds

        #x_test = (x_test - x_means)/x_stds
        #y_test = (y_test - y_means)/y_stds

        batch_size = len(x_train)

        fit_loss_train = np.zeros(num_epochs)
        best_net, best_loss = None, float('inf')
        nets = []

        for n in range(num_nets):
            net = Net_UCI(n_feature=in_dim, n_hidden=num_hidden, n_output=1, learn_rate=learn_rate, weight_decay=weight_decay)

            sub_idx = np.random.choice(np.arange(0, len(x_train)), size = (int(len(x_train)*data_fraction),), replace=True)
            x_train_sub, y_train_sub = x_train[sub_idx], y_train[sub_idx]

            for i in range(num_epochs):

                loss = net.fit(x_train_sub, y_train_sub)

                if log_every is not False and (i % log_every == 0 or i == num_epochs - 1) and len(nets) > 0:
                    train_loss, train_rmse = eval_ensemble(x_train, y_train, nets)
                    test_loss, test_rmse = eval_ensemble(x_test, y_test, nets)
                    print('Epoch %3d, network %2d, Loss train/test %.3f/%.3f, RMSE train/test %.3f/%.3f' % \
                          (i+1, len(nets), train_loss.cpu().data.numpy(), test_loss.cpu().data.numpy(),
                          train_rmse.cpu().data.numpy(), test_rmse.cpu().data.numpy()))

            nets.append(copy.deepcopy(net))


        train_loss, train_rmse = eval_ensemble(x_train, y_train, nets)
        test_loss, test_rmse = eval_ensemble(x_test, y_test, nets)

        train_logliks.append(-train_loss.cpu().data.numpy() - np.log(y_stds)[0])
        test_logliks.append(-test_loss.cpu().data.numpy() - np.log(y_stds)[0])

        train_rmses.append(y_stds[0]*train_rmse.cpu().data.numpy())
        test_rmses.append(y_stds[0]*test_rmse.cpu().data.numpy())

    print('Train log. lik. = %7.3f +/- %.3f' % (np.array(train_logliks).mean(), np.array(train_logliks).var()**0.5))
    print('Test  log. lik. = %7.3f +/- %.3f' % (np.array(test_logliks).mean(), np.array(test_logliks).var()**0.5))
    print('Train RMSE      = %7.3f +/- %.3f' % (np.array(train_rmses).mean(), np.array(train_rmses).var()**0.5))
    print('Test  RMSE      = %7.3f +/- %.3f' % (np.array(test_rmses).mean(), np.array(test_rmses).var()**0.5))
    
    return nets

# Housing dataset

In [19]:
batch_size = 64
data = pd.read_csv('/content/drive/MyDrive/bayesian-deep-learning-master/data/Datasets.csv').values

In [31]:
ensemble = train_ensemble(data=data, n_splits=10, num_epochs=100, num_nets=10, num_hidden=100,
                          learn_rate=1e-2, weight_decay=1e-2, data_fraction=0.8, log_every=False)

Train log. lik. =  -2.133 +/- 0.019
Test  log. lik. =  -2.142 +/- 0.099
Train RMSE      =   5.143 +/- 0.095
Test  RMSE      =   5.146 +/- 0.524


# Concrete compressive dataset

In [ ]:
np.random.seed(0)
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls" --no-check-certificate
data = pd.read_excel('Concrete_Data.xls', header=0, delimiter="\s+").values
data = data[np.random.permutation(np.arange(len(data)))]
data.shape

--2019-05-17 22:40:30--  https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 124928 (122K) [application/x-httpd-php]
Saving to: ‘Concrete_Data.xls.6’

Concrete_Data.xls.6 100%[===================>] 122.00K   290KB/s    in 0.4s    

2019-05-17 22:40:31 (290 KB/s) - ‘Concrete_Data.xls.6’ saved [124928/124928]



(1030, 9)

In [ ]:
ensemble = train_ensemble(data=data, n_splits=10, num_epochs=100, num_nets=20, num_hidden=100,
                          learn_rate=1e-1, weight_decay=0*1e-2, data_fraction=0.8, log_every=False)

Train log. lik. =  -2.849 +/- 0.085
Test  log. lik. =  -2.864 +/- 0.087
Train RMSE      =   9.078 +/- 0.481
Test  RMSE      =   9.407 +/- 0.969


# Energy efficiency dataset

In [ ]:
np.random.seed(0)
!wget "http://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx" --no-check-certificate
data = pd.read_excel('ENB2012_data.xlsx', header=0, delimiter="\s+").values
data = data[np.random.permutation(np.arange(len(data)))]
data.shape

--2019-05-17 22:14:45--  http://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76189 (74K) [application/x-httpd-php]
Saving to: ‘ENB2012_data.xlsx.1’

ENB2012_data.xlsx.1 100%[===================>]  74.40K   234KB/s    in 0.3s    

2019-05-17 22:14:46 (234 KB/s) - ‘ENB2012_data.xlsx.1’ saved [76189/76189]



(768, 10)

In [ ]:
ensemble = train_ensemble(data=data, n_splits=10, num_epochs=100, num_nets=20, num_hidden=100,
                          learn_rate=1e-2, weight_decay=1e-2, data_fraction=0.8, log_every=False)

Train log. lik. =  -1.490 +/- 0.113
Test  log. lik. =  -1.501 +/- 0.107
Train RMSE      =   2.633 +/- 0.121
Test  RMSE      =   2.638 +/- 0.269


# Power dataset

In [ ]:
np.random.seed(0)
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00294/CCPP.zip" --no-check-certificate 
zipped = zipfile.ZipFile("CCPP.zip")
data = pd.read_excel(zipped.open('CCPP/Folds5x2_pp.xlsx'), header=0, delimiter="\t").values
np.random.shuffle(data)
data.shape

--2019-05-17 22:15:13--  https://archive.ics.uci.edu/ml/machine-learning-databases/00294/CCPP.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3674852 (3.5M) [application/x-httpd-php]
Saving to: ‘CCPP.zip.1’

CCPP.zip.1          100%[===================>]   3.50M  3.13MB/s    in 1.1s    

2019-05-17 22:15:14 (3.13 MB/s) - ‘CCPP.zip.1’ saved [3674852/3674852]



(9568, 5)

In [ ]:
ensemble = train_ensemble(data=data, n_splits=10, num_epochs=100, num_nets=20, num_hidden=100,
                          learn_rate=1e-2, weight_decay=1e-2, data_fraction=0.8, log_every=False)

Train log. lik. =  -2.034 +/- 0.012
Test  log. lik. =  -2.038 +/- 0.020
Train RMSE      =   4.498 +/- 0.053
Test  RMSE      =   4.500 +/- 0.147


# Red wine dataset

In [ ]:
np.random.seed(0)
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv" --no-check-certificate 
data = pd.read_csv('winequality-red.csv', header=1, delimiter=';').values
data = data[np.random.permutation(np.arange(len(data)))]
data.shape

--2019-05-17 22:15:47--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84199 (82K) [application/x-httpd-php]
Saving to: ‘winequality-red.csv.2’

winequality-red.csv 100%[===================>]  82.23K   291KB/s    in 0.3s    

2019-05-17 22:15:47 (291 KB/s) - ‘winequality-red.csv.2’ saved [84199/84199]



(1598, 12)

In [ ]:
ensemble = train_ensemble(data=data, n_splits=10, num_epochs=100, num_nets=20, num_hidden=100,
                          learn_rate=1e-2, weight_decay=1e-2, data_fraction=0.8, log_every=False)

Train log. lik. =  -0.141 +/- 0.011
Test  log. lik. =  -0.152 +/- 0.086
Train RMSE      =   0.719 +/- 0.007
Test  RMSE      =   0.719 +/- 0.069


# Yacht dataset

In [ ]:
np.random.seed(0)
!wget "http://archive.ics.uci.edu/ml/machine-learning-databases/00243/yacht_hydrodynamics.data" --no-check-certificate 
data = pd.read_csv('yacht_hydrodynamics.data', header=1, delimiter='\s+').values
data = data[np.random.permutation(np.arange(len(data)))]
data.shape

--2019-05-17 22:16:19--  http://archive.ics.uci.edu/ml/machine-learning-databases/00243/yacht_hydrodynamics.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11487 (11K) [application/x-httpd-php]
Saving to: ‘yacht_hydrodynamics.data.2’

yacht_hydrodynamics 100%[===================>]  11.22K  --.-KB/s    in 0s      

2019-05-17 22:16:19 (311 MB/s) - ‘yacht_hydrodynamics.data.2’ saved [11487/11487]



(306, 7)

In [ ]:
ensemble = train_ensemble(data=data, n_splits=10, num_epochs=100, num_nets=20, num_hidden=100,
                          learn_rate=1e-2, weight_decay=1e-2, data_fraction=0.8, log_every=False)

Train log. lik. =  -2.503 +/- 0.112
Test  log. lik. =  -2.540 +/- 0.114
Train RMSE      =  11.073 +/- 0.250
Test  RMSE      =  10.882 +/- 2.943
